In [63]:
import os
import sys
import glob
import argparse
import signal

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as so

import astropy.units as u
from astropy.constants import c
from astropy.time import Time

from scipy.ndimage.filters import gaussian_filter

from dynspectools import read_psrflux
from ScintArcs import compute_nutSS, compute_staufD, ParabolicFitter, LineFinder

In [109]:
def plot_pano(dynspec, freqs, T, fref=1000., xlim=None, ylim=None, vm=3., 
              aspect=(10,10), plot_title=True, curv=0, normS=1, npad=0):

    """
    dynspec:  array with units [time, frequency]
    freqs: array of frequencies in MHz
    t0: Starting time in mjd.  Assumed 10 second bins
    xlim: xaxis limits in mHz
    ylim: yaxis limits in mus
    """
    
    # Get frequency and time info for plot axes
    
    t0 = Time(T[0], format='mjd', precision=0)
    dt = np.mean(np.diff(T[1:-1].unix))*u.s
    t = np.arange(dynspec.shape[0])*dt.value
    
    bw = freqs[-1] - freqs[0]
    df = (freqs[1]-freqs[0])*u.MHz
    dynspec = dynspec / np.std(dynspec)
    mn = np.mean(dynspec)
    nu = freqs*1e6
    
    # 2D power spectrum is the Secondary spectrum        
    nu0 = fref*1e+6 #or np.mean(nu)
    if npad:
        pad_width= ((npad*dynspec.shape[0], npad*dynspec.shape[0]), (0, 0))
        dynspec = np.pad(dynspec, pad_width, mode='constant')
        t = np.arange(dynspec.shape[0])*dt.value

    # Compute secondary spectrum
    ft,tau,SS = compute_nutSS(t,nu,dynspec,nu0=nu0)
    
    ft = ft * 1000.
    tau = tau * 1e6
    Sb = SS #- np.mean(SS[abs(ft)>50], axis=0, keepdims=True)
    Sb = np.log10(Sb)
    
    if normS:
        Ngrid = 1000
        taumin = 0.1
        Sb_norm = np.copy(Sb)
        Sb_norm[abs(ft)<0.1] = np.median(Sb_norm)
        Snorm, ftnormaxis, curvaxis = norm_sspec(Sb_norm, ft, tau, Ngrid=Ngrid, taumin=taumin)

    slow = np.median(Sb)-0.2
    shigh = slow + vm

    # Not the nicest, have a set of different plots it can produce
    plt.figure(figsize=aspect)
    if normS:
        ax2 = plt.subplot2grid((5, 2), (0, 0), rowspan=5)
        ax3 = plt.subplot2grid((5, 2), (3, 1), rowspan=2)
        ax4 = plt.subplot2grid((5, 2), (1, 1), rowspan=2)
        axprof = plt.subplot2grid((5, 2), (0, 1), rowspan=1)
    else:
        ax2 = plt.subplot2grid((2, 2), (0, 0), rowspan=2)
        ax3 = plt.subplot2grid((2, 2), (0, 1), rowspan=2)

    plt.subplots_adjust(wspace=0.1, hspace=0.02)

    # Plot dynamic spectrum image

    ax2.imshow(dynspec.T, aspect='auto', vmax=mn+7, vmin=mn-2, origin='lower',
                extent=[0,t[-1]/60., min(freqs), max(freqs)], cmap='viridis')
    ax2.set_xlabel('time (min)', fontsize=16)
    ax2.set_ylabel('freq (MHz)', fontsize=16)
    ax3.yaxis.tick_right()
    ax3.yaxis.set_label_position("right")

    # Plot Secondary spectrum
    ax3.imshow(Sb.T, aspect='auto', vmin=slow, vmax=shigh, origin='lower',
               extent=[min(ft), max(ft), min(tau), max(tau)], interpolation='nearest',
              cmap='viridis')
    ax3.set_xlabel(r'$f_{D}$ (mHz)', fontsize=16)
    ax3.set_ylabel(r'$\tau$ ($\mu$s)', fontsize=16) 

    if curv:
        plt.plot(ft, curv*ft**2.0, linestyle='dotted', color='tab:orange')
    
    if normS:
        logimg = np.copy(Snorm)
        logimg[abs(logimg)< 1e-9] = 0 #np.nan
        mask = np.zeros_like(logimg)
        mask[~np.isnan(logimg)] = 1

        xgrid = np.copy(ftnormaxis)

        powerprof = np.nanmean(logimg[:,(tau>taumin)], axis=-1)
        ic = np.nanmean(mask[:,(tau>taumin)], axis=-1)
        ic[ic<1e-3] = 1
        powerprof = powerprof / ic

        xsmooth = 0
        dxgrid = xgrid[1] - xgrid[0]
        xbin_smooth = int(xsmooth/dxgrid)
        if xsmooth:
            powerprof = powerprof - gaussian_filter(powerprof, xbin_smooth)

        ax4.imshow(Snorm.T, aspect='auto', vmin=slow, vmax=shigh, origin='lower',
               extent=[min(ft), max(ft), min(tau), max(tau)], interpolation='nearest',
              cmap='viridis')
        
        ax4.set_xticks([])
        ax4.set_ylim(0, max(tau))
        ax4.yaxis.tick_right()
        ax3.set_xlabel(r'$f_{D}$ (mHz)', fontsize=16)
        ax3.set_ylabel(r'$\tau$ ($\mu$s)', fontsize=16)
        axprof.set_xticks([])
        axprof.plot(ftnormaxis, powerprof)
        axprof.yaxis.tick_right()
        axprof.yaxis.set_label_position("right")

        axprof.set_xlim(-xlim*max(ft), xlim*max(ft) )
        ax4.set_xlim(-xlim*max(ft), xlim*max(ft) )

        
    if xlim:
        ax3.set_xlim(-xlim*max(ft), xlim*max(ft) )
    if normS:
        ax3.set_ylim(0, ylim*max(tau))
    else:
        ax3.set_ylim(ylim*min(tau), ylim*max(tau))
    if plot_title:
        #ax3.set_title(' {0}'.format(psrname), fontsize=18)
        ax2.set_title(' {0}'.format(t0.isot), fontsize=18)
        
    if normS:
        return SS, ft, tau, Snorm, ftnormaxis, curvaxis
    else:
        return CS, ft, tau


def norm_sspec(S, ft, tau, Ngrid=2000, taumin=0.1, tauref=None, xr=1.):
    
    img = np.zeros( (Ngrid+1, len(tau) ) )
    if not tauref:
        tauref = max(tau)
        
    xgrid = np.linspace(-xr*max(ft), xr*max(ft), Ngrid+1, endpoint=True)    
    for i in range(len(tau)):
        taui = tau[i]
        if (taui > taumin):
            for j in range(Ngrid+1):
                fti = xgrid[j]*np.sqrt(taui/ tauref)
                findex = find_nearest(ft, fti)
                img[j, i] = S[findex, i]
                
    curvaxis = tauref / (xgrid)**2.0
    return img, xgrid, curvaxis
                   
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def fit_parabola(x, curv, x0=0, C=0):
    return curv*(x-x0)**2 + C

def fit_curvature(Snorm, ftnormaxis, tauaxis, taulow=0.2, ftcut=1, plot=0, half=1):
    """
    Fit for arc curvature from normalized secondary spectrum
    
    Values hardcoded for use on Meerkat TPA dataset
    """
    
    logimg = np.copy(Snorm)
    logimg[abs(logimg)< 1e-9] = 0
    
    mask = np.zeros_like(logimg)
    mask[~np.isnan(logimg)] = 1
    
    xgrid = np.copy(ftnormaxis)

    taumax = max(tauaxis)
    xlim = max(ftnormaxis)

    powerprof = np.nanmean(logimg[:,(tauaxis>taulow)], axis=-1)
    ic = np.nanmean(mask[:,(tauaxis>taulow)], axis=-1)
    ic[ic<1e-3] = 1
    powerprof = powerprof / ic

    # average over +ve and -ve ftaxis, or take brighter half
    midprof = powerprof.shape[0]//2
    
    
    profL = powerprof[:midprof][::-1]
    profR = powerprof[midprof+1:]
    
    if half:
        if np.max(profL) > np.max(profR):
            curvprof = profL[:-1]
        else:
            curvprof = profR[1:]
    else:
        curvprof = powerprof[:midprof][::-1] + powerprof[midprof+1:]
    
    curvax2 = curvaxis[midprof+2:]
    ftprofaxis = ftnormaxis[midprof+2:]

    # Take peak as starting guess
    ftmax_index = np.argmax(curvprof)
    ft0 = ftprofaxis[ftmax_index]
    C0 = curvprof[ftmax_index]
    ftlim = 8

    fitrange = slice(  max( (ftmax_index-ftlim), 0), ftmax_index+ftlim+1)
    ft_fit = ftprofaxis[fitrange]
    prof_fit = curvprof[fitrange]
    
    p0 = [-0.1, ft0, C0]
    popt, pcov = so.curve_fit(fit_parabola, ft_fit, prof_fit, p0=p0, )

    tauref = taumax
    ftcurv = popt[1]
    ftcurverr = np.sqrt(pcov[1][1])

    curvavg = tauref / ftcurv**2.0
    curvmin = tauref / (ftcurv+ftcurverr)**2
    curvmax = tauref / (ftcurv-ftcurverr)**2
    curverr = (abs(curvavg-curvmax) + abs(curvavg-curvmin))/2.

    if plot:
        plt.figure(figsize=(5,3))
        plt.plot(ftprofaxis, curvprof)
        plt.plot(ft_fit, fit_parabola(ft_fit, *popt))
        plt.xlabel(r'$f_{D}$ (mHz)', fontsize=14)
        plt.ylabel('log10 power (arb)', fontsize=14)
        plt.xlim(0, max(ft_fit)*4)
        
    return curvavg, curverr, ftcurv, ftcurverr
    

In [ ]:
"""
Inputs for generating plots and fitting
"""

xlim = 0.2    # Fraction of f_t to plot
ylim = 1.     # Fraction of tau to plot
fref = 1000.  # Reference frequency for slowft in MHz
tmin = 1800.  # Minimum time of obs to consider
tpad = 5400.  # Pad all observations below this time
taulow = 0.1  # Zero all power below this tau in Norm secspec
vm = 3.       # Orders of magnitude in secspec plot
Flim = 1200.  # Crop all frequencies below this value

outfile = 'autocurvs.txt'
dspecfiles = np.sort(glob.glob('../J1439-5501/*dynspec'))


lam = c / (fref*u.MHz)
results = open(outfile, 'a')
results.write('name,mjd,freq,bw,tobs,dt,df,betaeta,betaetaerr\n')

for fn in dspecfiles:
    print("Fitting {0}".format(fn))

    # Load the dynamic spectrum from a .dynspec file
    DS, dynspec_err, T, Funits, source = read_psrflux(fn)
        
    # Setting up units
    F = Funits.to(u.MHz).value
    t = T.unix - T[0].unix
    nu = Funits.to(u.Hz).value
    dt = (t[2] - t[1])*u.s
    df = F[1] - F[0]
    bw = (max(F) - min(F)) + df
    
    # Restrict the frequency range, if necessary
    if max(F) > Flim:
        frange = np.argwhere(F < Flim).squeeze()
        DS = DS[:,frange]
        F = F[frange]
        nu = nu[frange]
    
    # Skip short observations
    Tobs = t[-1] + dt.value
    if Tobs < tmin:
        print("Skipping {0}, Tobs of {1} shorter than Tmin of {2}".format(fn, Tobs, tmin))
        continue

    # Pad short observations
    if Tobs < tpad:
        print("Tobs {0} shorter than {1}, padding with zeros".format(Tobs, tpad))
        npad = 1
    else:
        npad = 0
        
    nu0 = fref*1e+6 #or np.mean(nu)
    if npad:
        pad_width= ((npad*DS.shape[0], npad*DS.shape[0]), (0, 0))
        DS = np.pad(DS, pad_width, mode='constant')
        t = np.arange(DS.shape[0])*dt.value

    SS, ft, tau, Snorm, ftnormaxis, curvaxis = plot_pano(DS, F, T, xlim=xlim, ylim=ylim, vm=vm)   
    eta, eta_err, ftcurv, ftcurverr = fit_curvature(Snorm, ftnormaxis, tau, plot=1, taulow=taulow)
    
    print("eta={0} +- {1}".format(eta,eta_err))
    lamcurv = (eta*u.s**3* fref*u.MHz / lam).to(u.m**-1 * u.mHz**-2)
    lamcurverr = (eta_err*u.s**3* fref*u.MHz / lam).to(u.m**-1 * u.mHz**-2)
    results.write('{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}\n'.format(
        fn, T[0].mjd, fref, bw, t[-1], dt.value, df, lamcurv.value, lamcurverr.value))
    results.flush()


Fitting ../J1439-5501/J1439-5501_2019-04-16-23:05:36_zap.dynspec
Skipping ../J1439-5501/J1439-5501_2019-04-16-23:05:36_zap.dynspec, Tobs of 239.99399995803833 shorter than Tmin of 1800.0
Fitting ../J1439-5501/J1439-5501_2019-04-22-18:03:28_zap.dynspec
Skipping ../J1439-5501/J1439-5501_2019-04-22-18:03:28_zap.dynspec, Tobs of 511.9979999065399 shorter than Tmin of 1800.0
Fitting ../J1439-5501/J1439-5501_2019-07-01-14:53:52_zap.dynspec
Skipping ../J1439-5501/J1439-5501_2019-07-01-14:53:52_zap.dynspec, Tobs of 260.6580002307892 shorter than Tmin of 1800.0
Fitting ../J1439-5501/J1439-5501_2019-08-12-16:47:52_zap.dynspec
Skipping ../J1439-5501/J1439-5501_2019-08-12-16:47:52_zap.dynspec, Tobs of 516.8639998435974 shorter than Tmin of 1800.0
Fitting ../J1439-5501/J1439-5501_2019-12-27-11:27:20_zap.dynspec
Skipping ../J1439-5501/J1439-5501_2019-12-27-11:27:20_zap.dynspec, Tobs of 517.2360000610352 shorter than Tmin of 1800.0
Fitting ../J1439-5501/J1439-5501_2020-04-09-03:11:20_zap.dynspec
Skip

/home/ramain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:149: RuntimeWarning: divide by zero encountered in true_divide


eta=1.2614388577643472 +- 0.08977737981784384
Fitting ../J1439-5501/J1439-5501_2020-09-19-10:32:56_zap.dynspec
Tobs 2051.633999824524 shorter than 5400.0, padding with zeros


/home/ramain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:149: RuntimeWarning: divide by zero encountered in true_divide


eta=0.13865056416479277 +- 0.002413251253467724
Fitting ../J1439-5501/J1439-5501_2020-10-30-07:57:52_zap.dynspec
Tobs 2051.7599999904633 shorter than 5400.0, padding with zeros
